In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
import glob
from tqdm.notebook import tqdm
from time import sleep
from sklearn.metrics.pairwise import linear_kernel
from IPython.display import display

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# 設定列和欄的寬度
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
show=2

In [2]:
item_dict = {
    '國內股票開放型指數型':'ET000001',
    '國內股票開放型科技類':'ET001001',
    '國內股票開放型中小型':'ET001004',
    '國內股票開放型價值型':'ET001007',
    '跨國投資股票型區域型':'ET003002'}

In [3]:
def pprint(text, l=3):
    if l <= show:
        print(text)

In [4]:
def display_and_select(data_dict):
    while 1:
        for i, item in enumerate(data_dict.keys()):
            pprint(f"{i+1:>2}. {item}", l=0)
        sel = input("Please select: ")
        if sel.isdigit() and 0 < int(sel) <= len(data_dict):
            sel = int(sel) - 1
            break
        else:
            pprint('Wrong input!', l=0)
        
    return list(data_dict.keys())[sel], list(data_dict.values())[sel]

def select_type():
    selected_item, item_value = display_and_select(item_dict)
   
    pprint(f"Selected: {selected_item}", l=1)
    url = f'https://www.moneydj.com/funddj/ya/yp401000.djhtm?A={item_value}&B=901'
    pprint(url, l=1)
    return url

In [5]:
def fatch_fund_url(url):
    pprint(f"Fetching find data...", l=0)
    response = requests.get(url)
    if response.status_code != 200:
        pprint(f"Fatch_fund_url() return code: {response.status_code}", l=0)
    response.encoding = 'big5'
    soup = BeautifulSoup(response.text, "html.parser")
    target_table = soup.find("table", id="oMainTable")
    
    hyperlinks = []
    for row in target_table.findAll("tr"):
        cells = row.findAll("td")
        if cells:
            fund_name_cell = cells[1]  
            link = fund_name_cell.find("a")
            if link:
                hyperlinks.append([link.text, link["href"].split('?a=')[-1]])
                
    return hyperlinks
# print(hyperlinks)

In [6]:
def fetch_fund_coutent(fund_name, url):
    # url = 'https://www.moneydj.com/funddj/ya/yp401000.djhtm?A=ET000001&B=901'
    # url = 'https://www.moneydj.com/funddj/yp/yp013000.djhtm?a=AC0056'
    fund_id = url.split('=')[-1]
    response = requests.get(url)
    if response.status_code != 200:
        pprint(f"Return code: {response.status_code}", l=0)
    response.encoding = 'big5'
    soup = BeautifulSoup(response.text, "html.parser")

    df_list = []
    
    all_table = soup.find_all('table')

    for table_idx, table in enumerate(all_table):
        wanted = False
        has_data = True
        table_name = ''
        for td in table.find_all('td'):
            if '基金投資分佈' in td.text:
                table_name = td.text.strip()
                wanted = True
            if '無資料'in td.text:
                has_data = False
                pprint(f"{table_name} {td.text}", l=3)
                break

        if wanted and has_data:
            pprint(table, l=3)
            pprint(f'Found {table_name}', l=3)

            headers = [header.text for header in table.find_all("td", class_='t2c1')]
            headers +=  [header.text for header in table.find_all("td", class_='t2')]
            pprint(headers, l=3)

            def make_table(fund_id, fund_name, type_, content):  
                rows = content.find_all("tr")
                data = [[col.text.strip() for col in row.find_all("td")] for row in rows[1:]]

                df = pd.DataFrame(data, columns=headers)
                df_list.append(df)
                filename = f"{fund_id}_{fund_name}_{type_}.csv"
                pprint(f"Save {filename}.", l=1)
                df.to_csv(filename, index=False, encoding='utf-8-sig')

            for clas in ['t01', 't03']:
                sub_table = table.find('table', class_=clas)
                if not sub_table:
                    continue
                pprint(f"{table_name}, {len(sub_table)}", l=3)
                if not sub_table:
                    pprint(f"{fund_id} {table_name} NO list table!", l=1)
                    break
                if len(sub_table)>3 :
                    make_table(fund_id, fund_name, table_name, sub_table)
                    break
            else:
                pprint(f"{table_name} 無資料", l=3)
                    
#     print(df_list)
'''
在進行爬蟲時，爬取"依持有類股"的表格時遇到多的問題，一方面是要定位出我需要的表格，
另一方面是總共三種表格之中這個表格的class與前兩種不同，且中間還多了一個圓餅圖，
故在爬取時多了一些針對性的處理與判斷，未來網頁要是有做修改，那CODE也必須跟個做調整
'''

'\n在進行爬蟲時，爬取"依持有類股"的表格時遇到多的問題，一方面是要定位出我需要的表格，\n另一方面是總共三種表格之中這個表格的class與前兩種不同，且中間還多了一個圓餅圖，\n故在爬取時多了一些針對性的處理與判斷，未來網頁要是有做修改，那CODE也必須跟個做調整\n'

In [7]:
def fetch_fund_class():
    type_url = select_type()
    fund_url_list = fatch_fund_url(type_url)
    pprint(f"Funds: {len(fund_url_list)}", l=0)
    if len(fund_url_list) >30:
        sel = input("Go on to fetch for large files? (y/n)")
        if sel.lower() != 'y':
            print('Canceld')
            return False
    base_link = 'https://www.moneydj.com/funddj/yp/yp013000.djhtm?a='
    for j, i in enumerate(fund_url_list):
        content_link = base_link+i[1]
        pprint(f"{j:>3}. {i[0]}: \n\t{content_link}", l=1)
        fetch_fund_coutent(i[0], content_link)
    return True
        
# fetch_fund_class()

In [8]:
def merge_files():
    pprint(f"Merging files...", l=0)
    file_paths = glob.glob('*(依持有類股).csv')
    
    # Initialize an empty dataframe for concatenation
    merged_df = pd.DataFrame()

    # Loop through each file, read the content, add 'Fund_ID' and 'Fund_name' columns and concatenate
    for file_path in file_paths:
        fund_id = file_path.split('/')[-1].split('_')[0]  # Extract Fund_ID from file name
        fund_name = file_path.split('/')[-1].split('_')[1]
        temp_df = pd.read_csv(file_path)
        temp_df['Fund_ID'] = fund_id
        temp_df['Fund_name'] = fund_name
        merged_df = pd.concat([merged_df, temp_df], ignore_index=True)

    # Create a mapping from Fund_ID to Fund_name before grouping
    fund_name_mapping = merged_df[['Fund_ID', 'Fund_name']].drop_duplicates().set_index('Fund_ID').to_dict()['Fund_name']

    # Drop the '投資金額(以萬元為單位)' column
    merged_df.drop(columns=['投資金額(以萬元為單位)'], inplace=True)
    
    # If '比例(%)' column is of object type (possibly string), then remove commas
    if merged_df['比例(%)'].dtype == 'object':
        merged_df['比例(%)'] = pd.to_numeric(merged_df['比例(%)'].str.replace(',', ''), errors='coerce')

    # Group by 'Fund_ID' and '名稱' and sum the '比例(%)' values
    merged_df = merged_df.groupby(['Fund_ID', '名稱']).sum(numeric_only=True).reset_index()

    # Pivot the table
    pivoted_df = merged_df.pivot(index='Fund_ID', columns='名稱', values='比例(%)')
    pivoted_df.reset_index(inplace=True)
    
    # Add Fund_name column using the mapping
    pivoted_df['Fund_name'] = pivoted_df['Fund_ID'].map(fund_name_mapping)
    
    # Reorder columns
    columns_order = ['Fund_ID', 'Fund_name'] + [col for col in pivoted_df if col not in ['Fund_ID', 'Fund_name']]
    pivoted_df = pivoted_df[columns_order]

    # Fill NaN values with 0
    pivoted_df.fillna('0', inplace=True)
    
    pivoted_df.to_csv('Total.csv', index=False, encoding='utf-8-sig')
    
    return pivoted_df

# merge_files()


'''
在資料合併時是相對比較難處理的，因為每個基金的持股分類csv檔內容中有的欄位並不相同，因此以pd.concat來做合併是很重要的方法，
之後再將ID與名稱映射回合併的df，再把NaN的值填為零，就可以進行linear_kernel的計算。

'''

'\n在資料合併時是相對比較難處理的，因為每個基金的持股分類csv檔內容中有的欄位並不相同，因此以pd.concat來做合併是很重要的方法，\n之後再將ID與名稱映射回合併的df，再把NaN的值填為零，就可以進行linear_kernel的計算。\n\n'

In [9]:
def recommend_funds_with_detailed_distribution(df, fund_input, n_recommendations=5, n_largest=5):
    """
    Recommend similar funds based on the provided fund ID or fund name and also show their top 5 investment distribution 
    percentages in descending order.
    
    Parameters:
    - fund_input: Fund ID or Fund name for which recommendations are to be made.
    - n_recommendations: Number of top similar funds to recommend. Default is 5.
    
    Returns:
    - DataFrame containing recommended funds, their similarity scores, and detailed top 5 investment distribution.
    """
    investment_data = df.iloc[:, 2:]
    similarity_matrix = linear_kernel(investment_data, investment_data)

    # Find the index of the given fund
    idx = df[df['Fund_ID'] == fund_input].index[0]
    
    input_fund_distribution = df.iloc[idx, 2:].sort_values(ascending=False)
#     print("Top 5 investment categories for the input fund:")
    print(f"Top 5 investment categories for the fund <{fund_input}>:")
    print(input_fund_distribution.head(n_largest))
    
    # Get the pairwise similarity scores for the given fund
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    
    # Sort the funds based on similarity scores
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores for the top n similar funds
    top_scores = sorted_scores[1:n_recommendations+1]
    
    # Get the fund indices for the top n similar funds
    fund_indices = [i[0] for i in top_scores]
    
    # Create the recommendations DataFrame
    recommended_funds = df.iloc[fund_indices].copy()
    recommended_funds['Similarity_Score'] = [round(i[1],2) for i in top_scores]
    
    # Get the top 5 investment categories and their percentages for each recommended fund
    investment_cols = df.columns[2:-1]
    recommended_funds['Top_Investments'] = recommended_funds[investment_cols].apply(
        lambda row: ", ".join([f"{col} ({row[col]:.2f}%)" for col in row.nlargest(n_largest).index]), axis=1)
    
    final = recommended_funds[['Fund_ID', 'Fund_name', 'Similarity_Score', 'Top_Investments']].copy()
    final.to_csv('recommand.csv', index=False, encoding='utf-8-sig')
    
    return final

'''
最後在呈現結果時思考應該要列出推薦的五支基金各自的前五大持股分類，還是要五支基金綜合的前五大持股分類的各自比例，
故在推薦部分做了兩種，但是出乎意料的是直接撈出各自的持股比例來呈現反而是比較難處理的，
因為要從各個欄位擷取出前五大，再整合至一個欄位之中需要比較多的特別處理。
'''

'\n最後在呈現結果時思考應該要列出推薦的五支基金各自的前五大持股分類，還是要五支基金綜合的前五大持股分類的各自比例，\n故在推薦部分做了兩種，但是出乎意料的是直接撈出各自的持股比例來呈現反而是比較難處理的，\n因為要從各個欄位擷取出前五大，再整合至一個欄位之中需要比較多的特別處理。\n'

In [10]:
def recommend_funds_with_combined_distribution(df, fund_input, n_recommendations=5, n_largest=5):
    """
    Recommend similar funds based on the provided fund ID or fund name and also show their investment distribution 
    percentages in the top 5 investment categories of the combined recommended funds.
    """
    
    investment_data = df.iloc[:, 2:]
    similarity_matrix = linear_kernel(investment_data, investment_data)

    # Find the index of the given fund
    idx = df[df['Fund_ID'] == fund_input].index[0]
    
    input_fund_distribution = df.iloc[idx, 2:].sort_values(ascending=False)
    print(f"Top 5 investment categories for the fund <{fund_input}>:")
    print(input_fund_distribution.head(n_largest))
 
    # Get the pairwise similarity scores for the given fund
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    
    # Sort the funds based on similarity scores and get the top n similar funds
    top_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:n_recommendations+1]
    
    # Get the fund indices for the top n similar funds
    fund_indices = [i[0] for i in top_scores]
    
    # Create the recommendations DataFrame
    recommended_funds = df.iloc[fund_indices].copy()
    recommended_funds['Similarity_Score'] = [round(i[1],2) for i in top_scores]
    
    # Calculate the combined investment distribution for all recommended funds
    combined_distribution = recommended_funds.iloc[:, 2:-1].sum()
    
    # Get the top 5 investment categories from the combined distribution
    top_investments = combined_distribution.nlargest(n_largest).index.tolist()
    
    final = recommended_funds[['Fund_ID', 'Fund_name', 'Similarity_Score'] + top_investments]
    final.to_csv('recommend_combined.csv', index=False, encoding='utf-8-sig')
    
    return final

In [11]:
def suggesting(type_=1):
    # Load the CSV data
    df = pd.read_csv('Total.csv')

    id_list = list(df['Fund_ID'])
    name_list = list(df['Fund_name'])

    while 1:
        for i, j in enumerate(zip(id_list, name_list)):
            print(f"{i+1:>2}. {j[0]:>8}: {j[1]}")
        sel = input("Please select: ")
        if sel.isdigit() and 0 < int(sel) <= len(id_list):
            sel = int(sel) - 1
            break
        else:
            print('Wrong input!')

    pprint(f"Selection: {id_list[sel]} - {name_list[sel]}", l=0)
    if type_==1 :
        recommendations = recommend_funds_with_detailed_distribution(df, id_list[sel])
        
    else:
        recommendations = recommend_funds_with_combined_distribution(df, id_list[sel])
        
    return recommendations

In [12]:
def clean_csv():
    csv_files = glob.glob('*.csv')
    if len(csv_files) == 0:
        return
    pprint('Clean CSV files...', l=1)
    for file in tqdm(csv_files, desc="刪除檔案"):
        try:
            os.remove(file)
            pprint(f"已刪除檔案: {file}", l=1)
        except Exception as e:
            print(f"刪除檔案 {file} 時出錯: {e}")

In [ ]:
show=0 # 0~2 調整顯示訊息
sugg_type=0 # 1,0 調整推薦基金的持股比例顯示方式

def go():
    clean_csv()
    if not fetch_fund_class(): return
    merge_files()
    recomm = suggesting(sugg_type)
    display(recomm)
    
go()

刪除檔案:   0%|          | 0/28 [00:00<?, ?it/s]

 1. 國內股票開放型指數型
 2. 國內股票開放型科技類
 3. 國內股票開放型中小型
 4. 國內股票開放型價值型
 5. 跨國投資股票型區域型
